In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("./dados/vendas-por-fatura.csv")
df

,N° da fatura,Data da fatura,ID Cliente,País,Quantidade,Valor
0,548370,3/30/2021 16:14:00,15528.0,United Kingdom,123,"229,33"
1,575767,11/11/2021 11:11:00,17348.0,United Kingdom,163,"209,73"
2,C570727,10/12/2021 11:32:00,12471.0,Germany,-1,"-1,45"
3,549106,4/6/2021 12:08:00,17045.0,United Kingdom,1,"39,95"
4,573112,10/27/2021 15:33:00,16416.0,United Kingdom,357,"344,83"
...,...,...,...,...,...,...
25948,C556518,6/13/2021 10:53:00,16794.0,United Kingdom,-2,"-12,70"
25949,553060,5/13/2021 10:44:00,15826.0,United Kingdom,488,"720,00"
25950,577049,11/17/2021 13:58:00,17397.0,United Kingdom,104,"298,78"
25951,550122,4/14/2021 12:39:00,12931.0,United Kingdom,1176,"1488,00"


In [3]:
df.shape

(25953, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25953 entries, 0 to 25952
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   N° da fatura    25953 non-null  object 
 1   Data da fatura  25953 non-null  object 
 2   ID Cliente      22229 non-null  float64
 3   País            25953 non-null  object 
 4   Quantidade      25953 non-null  int64  
 5   Valor           25953 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.2+ MB


In [5]:
df.dtypes

N° da fatura       object
Data da fatura     object
ID Cliente        float64
País               object
Quantidade          int64
Valor              object
dtype: object

In [6]:
# Renomeando as colunas
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['N°_da_fatura', 'Data_da_fatura', 'ID_Cliente', 'País', 'Quantidade',
       'Valor'],
      dtype='object')

In [7]:
df = df.dropna(subset=["ID_Cliente"])
df

,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
0,548370,3/30/2021 16:14:00,15528.0,United Kingdom,123,"229,33"
1,575767,11/11/2021 11:11:00,17348.0,United Kingdom,163,"209,73"
2,C570727,10/12/2021 11:32:00,12471.0,Germany,-1,"-1,45"
3,549106,4/6/2021 12:08:00,17045.0,United Kingdom,1,"39,95"
4,573112,10/27/2021 15:33:00,16416.0,United Kingdom,357,"344,83"
...,...,...,...,...,...,...
25947,C553519,5/17/2021 14:16:00,18218.0,United Kingdom,-2,"-9,90"
25948,C556518,6/13/2021 10:53:00,16794.0,United Kingdom,-2,"-12,70"
25949,553060,5/13/2021 10:44:00,15826.0,United Kingdom,488,"720,00"
25950,577049,11/17/2021 13:58:00,17397.0,United Kingdom,104,"298,78"


In [8]:
# Convertendo para tipo inteiro o ID
df['ID_Cliente'] = df['ID_Cliente'].astype(int)

C:\Users\carla\AppData\Local\Temp/ipykernel_36228/1992455318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID_Cliente'] = df['ID_Cliente'].astype(int)


In [9]:
# Convertendo tipo da Data da Fatura
df["Data_da_fatura"] = pd.to_datetime(df["Data_da_fatura"])
df.head()


C:\Users\carla\AppData\Local\Temp/ipykernel_36228/4280545746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Data_da_fatura"] = pd.to_datetime(df["Data_da_fatura"])


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
0,548370,2021-03-30 16:14:00,15528,United Kingdom,123,"229,33"
1,575767,2021-11-11 11:11:00,17348,United Kingdom,163,"209,73"
2,C570727,2021-10-12 11:32:00,12471,Germany,-1,"-1,45"
3,549106,2021-04-06 12:08:00,17045,United Kingdom,1,"39,95"
4,573112,2021-10-27 15:33:00,16416,United Kingdom,357,"344,83"


In [10]:
# Ordenando ascendente pela Data da Fatura
df.sort_values(by=['Data_da_fatura'], inplace=True)
df.head(10)


c:\Users\carla\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,"139,12"
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,"22,20"
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,"70,05"
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,"278,73"
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,"17,85"
505,536370,2020-12-01 08:45:00,12583,France,449,"855,86"
12932,536371,2020-12-01 09:00:00,13748,United Kingdom,80,"204,00"
2773,536372,2020-12-01 09:01:00,17850,United Kingdom,12,"22,20"
20456,536373,2020-12-01 09:02:00,17850,United Kingdom,88,"259,86"
24617,536374,2020-12-01 09:09:00,15100,United Kingdom,32,"350,40"


In [11]:
# Obtendo Ano e Mês
df['Ano_mes'] = df['Data_da_fatura'].dt.to_period('M')
df.head(10)


C:\Users\carla\AppData\Local\Temp/ipykernel_36228/577416114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ano_mes'] = df['Data_da_fatura'].dt.to_period('M')


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,"139,12",2020-12
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,"22,20",2020-12
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,"70,05",2020-12
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,"278,73",2020-12
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,"17,85",2020-12
505,536370,2020-12-01 08:45:00,12583,France,449,"855,86",2020-12
12932,536371,2020-12-01 09:00:00,13748,United Kingdom,80,"204,00",2020-12
2773,536372,2020-12-01 09:01:00,17850,United Kingdom,12,"22,20",2020-12
20456,536373,2020-12-01 09:02:00,17850,United Kingdom,88,"259,86",2020-12
24617,536374,2020-12-01 09:09:00,15100,United Kingdom,32,"350,40",2020-12


In [12]:
df.loc[df['País']=="United Kingdom", 'Pertence_UK'] = "SIM"
df.loc[df['País']!="United Kingdom", 'Pertence_UK'] = "NÃO"
df.head(50)

c:\Users\carla\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\carla\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes,Pertence_UK
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,"139,12",2020-12,SIM
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,"22,20",2020-12,SIM
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,"70,05",2020-12,SIM
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,"278,73",2020-12,SIM
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,"17,85",2020-12,SIM
505,536370,2020-12-01 08:45:00,12583,France,449,"855,86",2020-12,NÃO
12932,536371,2020-12-01 09:00:00,13748,United Kingdom,80,"204,00",2020-12,SIM
2773,536372,2020-12-01 09:01:00,17850,United Kingdom,12,"22,20",2020-12,SIM
20456,536373,2020-12-01 09:02:00,17850,United Kingdom,88,"259,86",2020-12,SIM
24617,536374,2020-12-01 09:09:00,15100,United Kingdom,32,"350,40",2020-12,SIM


In [13]:
df.loc[df['N°_da_fatura'].str.startswith('C'), 'Devolucao'] = "SIM"
df.loc[~df['N°_da_fatura'].str.startswith('C'), 'Devolucao'] = "NÃO"
df.head(20)


c:\Users\carla\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\carla\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,N°_da_fatura,Data_da_fatura,ID_Cliente,País,Quantidade,Valor,Ano_mes,Pertence_UK,Devolucao
9367,536365,2020-12-01 08:26:00,17850,United Kingdom,40,"139,12",2020-12,SIM,NÃO
18259,536366,2020-12-01 08:28:00,17850,United Kingdom,12,"22,20",2020-12,SIM,NÃO
11185,536368,2020-12-01 08:34:00,13047,United Kingdom,15,"70,05",2020-12,SIM,NÃO
6876,536367,2020-12-01 08:34:00,13047,United Kingdom,83,"278,73",2020-12,SIM,NÃO
8195,536369,2020-12-01 08:35:00,13047,United Kingdom,3,"17,85",2020-12,SIM,NÃO
505,536370,2020-12-01 08:45:00,12583,France,449,"855,86",2020-12,NÃO,NÃO
12932,536371,2020-12-01 09:00:00,13748,United Kingdom,80,"204,00",2020-12,SIM,NÃO
2773,536372,2020-12-01 09:01:00,17850,United Kingdom,12,"22,20",2020-12,SIM,NÃO
20456,536373,2020-12-01 09:02:00,17850,United Kingdom,88,"259,86",2020-12,SIM,NÃO
24617,536374,2020-12-01 09:09:00,15100,United Kingdom,32,"350,40",2020-12,SIM,NÃO


In [14]:
print(df['ID_Cliente'].unique())

[17850 13047 12583 ... 13298 14569 12713]


In [15]:
# Verificando se há valores nulos
temp = df.isnull().values.any()


In [23]:
# contando clientes únicos por ano/mês
df_coorte = pd.DataFrame(df.groupby(['Ano_mes'])['ID_Cliente'].nunique()).reset_index()
df_coorte

,Ano_mes,ID_Cliente
0,2020-12,948
1,2021-01,783
2,2021-02,798
3,2021-03,1020
4,2021-04,899
5,2021-05,1079
6,2021-06,1051
7,2021-07,993
8,2021-08,980
9,2021-09,1302


,Ano_mes,Pertence_UK,Devolucao,ID_Cliente
0,2020-12,NÃO,NÃO,70
1,2020-12,NÃO,SIM,21
2,2020-12,SIM,NÃO,815
3,2020-12,SIM,SIM,229
4,2021-01,NÃO,NÃO,94
5,2021-01,NÃO,SIM,26
6,2021-01,SIM,NÃO,647
7,2021-01,SIM,SIM,177
8,2021-02,NÃO,NÃO,79
9,2021-02,NÃO,SIM,27
